In [1]:
import time
import glob
import os
import re
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

print(f'pandas: {pd.__version__}')
print(f'numpy: {np.__version__}')
print(f'matplotlib: {matplotlib.__version__}')
print(f'seaborn: {sns.__version__}')

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', str)

pandas: 2.1.1
numpy: 1.26.0
matplotlib: 3.8.0
seaborn: 0.13.0


# Load Access to gardens and public green space in Great Britain

The data is available from the [Office for National Statistics website](https://www.ons.gov.uk/economy/environmentalaccounts/datasets/accesstogardensandpublicgreenspaceingreatbritain) and is licensed under the [Open Government Licence 3.0](https://data.gov.ie/open-government-licence/). The data was last updated in 2020 but should be a sufficient guide for the purposes of this project.

As the data was produced as a result of Census 2011, it is only available at the [Middle Layer Super Output Areas (MSOAs) and Lower Super Output Area (LSOA)](https://ocsi.uk/2019/03/18/lsoas-leps-and-lookups-a-beginners-guide-to-statistical-geographies/) level which can contain update to 100 postcodes. But fortunately the data is split between houses and flats so we can look at the property type for each record and enrich them with more appropriate data accordingly. And again, this is only a rough estimation but is enough to tell us the surrounding area of the property.

In [3]:
def load_private_outdoor_space_data():
    df = pd.read_excel('./data/input/osprivateoutdoorspacereferencetables.xlsx', header=1, sheet_name=4)

    # Only keep the columns we need
    df = df.iloc[:, [6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20]]
    df.columns = ['MSOA', 'MSOAName', 'HouseWithPOS', 'HouseTotalPOS', 'HouseWithPOSPct', 'HouseAvgPOS', 'HouseMedPOS', 'FlatWithPOS', 'FlatTotalPOS', 'FlatPOSCount', 'FlatWithPOSPct', 'FlatAvgPOS', 'FlatPOSShare']

    df.set_index('MSOA', inplace=True)
    df = df.add_prefix('PGN_')
    return df

private_space_df = load_private_outdoor_space_data()
private_space_df.head()


,PGN_MSOAName,PGN_HouseWithPOS,PGN_HouseTotalPOS,PGN_HouseWithPOSPct,PGN_HouseAvgPOS,PGN_HouseMedPOS,PGN_FlatWithPOS,PGN_FlatTotalPOS,PGN_FlatPOSCount,PGN_FlatWithPOSPct,PGN_FlatAvgPOS,PGN_FlatPOSShare
MSOA,,,,,,,,,,,,
E02000001,City of London 001,20.0,380.0,0.27,19.0,16.3,534.0,3294.4,13.4,0.07,183.0,39.9
E02000002,Barking and Dagenham 001,1698.0,1199810.9,0.983,706.6,164.6,595.0,36880.3,165.4,0.481,245.9,3.6
E02000003,Barking and Dagenham 002,2634.0,433154.6,0.992,164.4,124.9,866.0,69334.0,323.8,0.612,222.9,2.7
E02000004,Barking and Dagenham 003,2037.0,389959.2,0.993,191.4,174.0,174.0,16058.4,98.1,0.513,214.1,1.8
E02000005,Barking and Dagenham 004,3089.0,410321.2,0.992,132.8,119.5,385.0,27837.8,119.2,0.706,163.8,3.2


In [5]:
def load_public_green_space_data():
    df = pd.read_excel('./data/input/ospublicgreenspacereferencetables.xlsx', header=0, sheet_name=7)

    # Only keep the columns we need
    df = df.iloc[:, 8:16]
    df.columns = ['LSOA', 'LSOAName', 'IMD', 'IMDDecile', 'NearestParkDistanceAvg', 'NearestParkSizeAvg', '1kParkCountAvg', '1kParkSizeAvg']

    df.set_index('LSOA', inplace=True)
    df = df.add_prefix('PGN_')
    return df

green_space_df = load_public_green_space_data()
green_space_df.head()

,PGN_LSOAName,PGN_IMD,PGN_IMDDecile,PGN_NearestParkDistanceAvg,PGN_NearestParkSizeAvg,PGN_1kParkCountAvg,PGN_1kParkSizeAvg
LSOA,,,,,,,
E01000001,City of London 001A,29199.0,9.0,183.6,2862.51,10.41,24104.56
E01000002,City of London 001B,30379.0,10.0,332.71,3110.62,9.99,18466.43
E01000003,City of London 001C,14915.0,5.0,153.09,3537.77,9.53,30942.24
E01000005,City of London 001E,8678.0,3.0,164.26,603.0,12.24,47134.26
E01000006,Barking and Dagenham 016A,14486.0,5.0,246.65,39927.22,4.57,474258.5


# Enrich Merged Price Paid Data

We will add the private outdoor space data to the merged price paid data using MOSA code and the property type (`EPC_PROPERTY_TYPE`), and add the public outdoor space data use the LSOA code.

In [6]:
from modules.utils import load_saved_data

ppd_df = load_saved_data('1_ppd_epc_data')

Loading saved data from ./data/saved/1_ppd_epc_data.parquet...


In [7]:
def enrich_with_green_space_data(ppd_df, private_space_df, green_space_df):
    ppd_df = ppd_df.reset_index()[['PPD_ID', 'EPC_PROPERTY_TYPE', 'ONSUD_MSOA', 'ONSUD_LSOA']]

    # Merge private_space_df and green_space_df in a single step
    ppd_df = ppd_df.merge(private_space_df, how='left', left_on='ONSUD_MSOA', right_index=True)
    ppd_df = ppd_df.merge(green_space_df, how='left', left_on='ONSUD_LSOA', right_index=True)

    ppd_df.drop(columns=[
        'EPC_PROPERTY_TYPE', 'ONSUD_MSOA', 'ONSUD_LSOA',
        'PGN_HouseWithPOSPct', 'PGN_HouseAvgPOS', 'PGN_HouseMedPOS', 'PGN_FlatWithPOSPct', 'PGN_FlatAvgPOS',
        'PGN_LSOAName', 'PGN_IMDDecile', 'PGN_MSOAName'
        ], inplace=True)

    # Set the index to PPD_ID
    ppd_df.set_index('PPD_ID', inplace=True)

    return ppd_df

enriched_ppd_df = enrich_with_green_space_data(ppd_df, private_space_df, green_space_df)

In [8]:
enriched_ppd_df[enriched_ppd_df['PGN_HouseWithPOS'].isna()]
# private_space_df.loc['E02006992']

,PGN_HouseWithPOS,PGN_HouseTotalPOS,PGN_FlatWithPOS,PGN_FlatTotalPOS,PGN_FlatPOSCount,PGN_FlatPOSShare,PGN_IMD,PGN_NearestParkDistanceAvg,PGN_NearestParkSizeAvg,PGN_1kParkCountAvg,PGN_1kParkSizeAvg
PPD_ID,,,,,,,,,,,
{BEF7EBBF-5F1B-7A76-E053-6B04A8C092F7},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{BEF7EBBF-5F1B-7A76-E053-6B04A8C092F7},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{BEF7EBBF-5F1B-7A76-E053-6B04A8C092F7},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{BEF7EBBF-5F1B-7A76-E053-6B04A8C092F7},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{BEF7EBBF-5F1B-7A76-E053-6B04A8C092F7},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
{D707E536-8794-0AD9-E053-6B04A8C067CC},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{D707E536-8794-0AD9-E053-6B04A8C067CC},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
{D707E536-8794-0AD9-E053-6B04A8C067CC},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import modules.dataset_onsud as onsud

onsud_df = onsud.load_data(london_only=True)
onsud_df[onsud_df['ONSUD_MSOA'] == 'E02006992']

Loading saved data from ./data/saved/saved_onsud_feb_2023.parquet...


,ONSUD_REGION,ONSUD_POSTCODE,ONSUD_BOROUGH,ONSUD_BOROUGH_CODE,ONSUD_WARD,ONSUD_WARD_CODE,ONSUD_OA,ONSUD_MSOA,ONSUD_LSOA
ONSUD_UPRN,,,,,,,,,
10010200757,E12000007,SE10 9PD,Greenwich,E09000011,East Greenwich,E05014076,E00182215,E02006992,E01034192
10010200758,E12000007,SE10 9PD,Greenwich,E09000011,East Greenwich,E05014076,E00182215,E02006992,E01034192
10010200764,E12000007,SE10 9PD,Greenwich,E09000011,East Greenwich,E05014076,E00182215,E02006992,E01034192
10010200772,E12000007,SE10 9NP,Greenwich,E09000011,East Greenwich,E05014076,E00182215,E02006992,E01034192
10010200773,E12000007,SE10 9NP,Greenwich,E09000011,East Greenwich,E05014076,E00182215,E02006992,E01034192
...,...,...,...,...,...,...,...,...,...
200001910962,E12000007,SE10 9NL,Greenwich,E09000011,East Greenwich,E05014076,E00008302,E02006992,E01034192
200001910963,E12000007,SE10 9NL,Greenwich,E09000011,East Greenwich,E05014076,E00008302,E02006992,E01034192
200002800572,E12000007,SE10 0NW,Greenwich,E09000011,Greenwich Peninsula,E05014082,E00182360,E02006992,E01034188


In [10]:
from modules.utils import save_data

# Save it for later
save_data(enriched_ppd_df, '5_ppd_green_space_data')

Saving data to ./data/saved/5_ppd_green_space_data.parquet...
